# Customized Graph Kernel with Fast GRF

This notebook demonstrates the use of a custom GPyTorch kernel based on the Fast Graph Random Features (GRF) approach with sparse implementations.

In [1]:
import torch
import gpytorch
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import laplacian
import networkx as nx
from sklearn.metrics import mean_squared_error
import sys
import os

# Add paths for importing the custom kernel
sys.path.append('/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs')
sys.path.append('/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs/efficient_graph_gp_sparse')

from efficient_graph_gp_sparse.gptorch_kernels_sparse.general_kernel_fast_grf import GraphGeneralFastGRFKernel

In [2]:
# Generate ring graph (same as test.ipynb)
np.random.seed(42)
torch.manual_seed(42)

n_nodes = 50

# Create ring graph (cycle graph)
G = nx.cycle_graph(n_nodes)
A = nx.adjacency_matrix(G)

# Ensure adjacency matrix is in CSR format
A = A.tocsr()

# Generate function that respects ring topology
# Use angular position on the ring for a smooth function
angles = np.linspace(0, 2*np.pi, n_nodes, endpoint=False)
true_func = lambda theta: 2*np.sin(2*theta) + 0.5*np.cos(4*theta) + 0.3*np.sin(theta)
y_true = true_func(angles)

# Add noise
noise_std = 0.1
y_observed = y_true + np.random.normal(0, noise_std, n_nodes)

# Training/test split
train_idx = np.random.choice(n_nodes, size=int(0.7 * n_nodes), replace=False)
test_idx = np.setdiff1d(np.arange(n_nodes), train_idx)

X_train = torch.tensor(train_idx, dtype=torch.float32).unsqueeze(1)
y_train = torch.tensor(y_observed[train_idx], dtype=torch.float32)
X_test = torch.tensor(test_idx, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_observed[test_idx], dtype=torch.float32)

print(f"Ring graph: {n_nodes} nodes, {G.number_of_edges()} edges")
print(f"Adjacency matrix format: {type(A)}")
print(f"Adjacency matrix shape: {A.shape}")
print(f"Adjacency matrix sparsity: {A.nnz / (n_nodes * n_nodes):.4f}")
print(f"Training set: {len(train_idx)} nodes")
print(f"Test set: {len(test_idx)} nodes")

Ring graph: 50 nodes, 50 edges
Adjacency matrix format: <class 'scipy.sparse._csr.csr_array'>
Adjacency matrix shape: (50, 50)
Adjacency matrix sparsity: 0.0400
Training set: 35 nodes
Test set: 15 nodes


In [3]:
class GraphGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, adjacency_matrix):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        
        # Use ScaleKernel to add output scaling to our custom kernel
        # Pass sparse adjacency matrix directly (kernel will handle conversion)
        base_kernel = GraphGeneralFastGRFKernel(
            adjacency_matrix=adjacency_matrix,  # Pass sparse matrix directly
            walks_per_node=30,
            p_halt=0.1,
            max_walk_length=5,
            random_walk_seed=42
        )
        self.covar_module = gpytorch.kernels.ScaleKernel(base_kernel)
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GraphGPModel(X_train, y_train, likelihood, A)

print("Model initialized successfully!")
print(f"Modulator vector shape: {model.covar_module.base_kernel.modulator_vector.shape}")
print(f"Number of step matrices: {len(model.covar_module.base_kernel.step_matrices)}")
print(f"Step matrix 0 shape: {model.covar_module.base_kernel.step_matrices[0].shape}")
print(f"Step matrix 0 sparsity: {model.covar_module.base_kernel.step_matrices[0]._nnz() / (n_nodes * n_nodes):.4f}")

/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs/efficient_graph_gp_sparse/gptorch_kernels_sparse/general_kernel_fast_grf.py:55: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  sparse_tensor = torch.sparse_csr_tensor(


Model initialized successfully!
Modulator vector shape: torch.Size([5])
Number of step matrices: 5
Step matrix 0 shape: torch.Size([50, 50])
Step matrix 0 sparsity: 0.0200


In [4]:
# Training
model.train()
likelihood.train()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 100
for i in range(training_iter):
    optimizer.zero_grad()
    output = model(X_train)
    loss = -mll(output, y_train)
    loss.backward()
    
    if i % 20 == 0:
        print(f'Iter {i+1}/{training_iter} - Loss: {loss.item():.3f} - '
              f'Output scale: {model.covar_module.outputscale.item():.3f} - '
              f'Noise: {likelihood.noise.item():.3f}')
        print(f'Modulator vector: {model.covar_module.base_kernel.modulator_vector.data}')
    
    optimizer.step()

RuntimeError: Calling add on a sparse CPU tensor requires compiling PyTorch with MKL. Please use PyTorch built MKL support.

In [ ]:
# Prediction
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    # Predict on test nodes
    test_pred = likelihood(model(X_test))
    test_mean = test_pred.mean
    test_std = test_pred.stddev
    
    # Predict on all nodes for visualization
    X_all = torch.arange(n_nodes, dtype=torch.float32).unsqueeze(1)
    all_pred = likelihood(model(X_all))
    all_mean = all_pred.mean
    all_std = all_pred.stddev

# Calculate metrics
test_rmse = np.sqrt(mean_squared_error(y_test.numpy(), test_mean.numpy()))
print(f"\nTest RMSE: {test_rmse:.4f}")
print(f"Test noise std: {noise_std:.4f}")

In [ ]:
# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: GP predictions vs true function
ax1.scatter(train_idx, y_train.numpy(), color='blue', alpha=0.6, label='Training data', s=30)
ax1.scatter(test_idx, y_test.numpy(), color='red', alpha=0.6, label='Test data', s=30)
ax1.plot(range(n_nodes), y_true, 'k--', label='True function', alpha=0.8)
ax1.plot(range(n_nodes), all_mean.numpy(), 'g-', label='GP mean', alpha=0.8)
ax1.fill_between(range(n_nodes), 
                 all_mean.numpy() - 2*all_std.numpy(),
                 all_mean.numpy() + 2*all_std.numpy(),
                 alpha=0.2, color='green', label='95% confidence')
ax1.set_xlabel('Node index')
ax1.set_ylabel('Function value')
ax1.set_title('Fast GRF Kernel GP Regression on Graph')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Graph structure (ring layout)
pos = {}
for i in range(n_nodes):
    angle = 2 * np.pi * i / n_nodes
    pos[i] = (np.cos(angle), np.sin(angle))

nx.draw(G, pos, ax=ax2, node_color=y_observed, node_size=80, 
        cmap='viridis', with_labels=False, edge_color='gray', alpha=0.8, width=2)
ax2.set_title('Ring Graph (colored by function values)')
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

print(f"\nLearned parameters:")
print(f"Modulator vector: {model.covar_module.base_kernel.modulator_vector.data}")
print(f"Output scale: {model.covar_module.outputscale.item():.4f}")
print(f"Noise variance: {likelihood.noise.item():.4f}")

In [ ]:
# Analyze the learned modulator vector
modulator = model.covar_module.base_kernel.modulator_vector.data.numpy()

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.bar(range(len(modulator)), modulator)
plt.xlabel('Walk step')
plt.ylabel('Modulator weight')
plt.title('Learned Modulator Vector')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.bar(range(len(modulator)), np.abs(modulator))
plt.xlabel('Walk step')
plt.ylabel('|Modulator weight|')
plt.title('Absolute Values of Modulator Weights')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Modulator vector statistics:")
print(f"Mean: {np.mean(modulator):.4f}")
print(f"Std: {np.std(modulator):.4f}")
print(f"Max: {np.max(modulator):.4f}")
print(f"Min: {np.min(modulator):.4f}")

In [ ]:
# Check PyTorch version and sparse tensor support
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Test sparse tensor creation
try:
    # Create a simple sparse tensor to test functionality
    indices = torch.LongTensor([[0, 1], [1, 0]])
    values = torch.FloatTensor([1.0, 2.0])
    sparse_test = torch.sparse_coo_tensor(indices.t(), values, (2, 2))
    print("✓ Sparse COO tensors working")
    
    # Test CSR tensor (requires PyTorch >= 1.9)
    crow_indices = torch.LongTensor([0, 1, 2])
    col_indices = torch.LongTensor([1, 0])
    values = torch.FloatTensor([1.0, 2.0])
    sparse_csr_test = torch.sparse_csr_tensor(crow_indices, col_indices, values, (2, 2))
    print("✓ Sparse CSR tensors working")
    
    print("All sparse tensor functionality is available!")
    
except Exception as e:
    print(f"❌ Sparse tensor error: {e}")
    print("You may need to upgrade PyTorch: pip install torch>=1.9.0")